In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import rasterio
from PIL import Image
from ultralytics import YOLO
import time
Image.MAX_IMAGE_PIXELS = None
import torch
import torchvision.transforms.functional as TF

In [3]:
import rasterio

# Open the raster file
raster_file = "C:/Users/rutvijw/Desktop/LabWork/geolocate/soybeanragweed2/soybeanragweed2/3_dsm_ortho/2_mosaic/SoybeanRagweed2_transparent_mosaic_group1.tif"
with rasterio.open(raster_file) as src:
    # Get the width and height of the raster file in its coordinate system
    width = src.width
    height = src.height

    # Get the resolution of the raster file
    transform = src.transform
    x_res = transform[0]
    y_res = -transform[4]  # Negative because y-coordinate increases downward

    # Calculate the area in square meters
    area = width * x_res * height * y_res

print("Area of the raster file:", area/4047, "acres")

Area of the raster file: 1.3480080352792192 acres


In [34]:
print("Area of one pixel:", src.res[0]*src.res[1], "m2")
print("Area of one pixel:", src.res[0]*src.res[1]/4047, "acre")
print("Area of one pixel:", src.res[0]*src.res[1]*10000, "cm2")
print("Area of one pixel:", src.res[0]*src.res[1]*10000*100, "mm2")

Area of one pixel: 2.1025e-06 m2
Area of one pixel: 5.195206325673339e-10 acre
Area of one pixel: 0.021025000000000002 cm2
Area of one pixel: 2.1025 mm2


In [10]:
src.res

(0.0014500000000000001, 0.0014500000000000001)

In [18]:
640 * 640 * 2.1025e-06

0.8611840000000001

In [12]:
1.732/0.00021279565109957996

8139.264082936979

In [13]:
8.1392640829369788946

8.139264082936979

In [7]:
import rasterio
from rasterio import mask
import numpy as np

# Open the raster file
raster_file = "C:/Users/rutvijw/Desktop/geolocate/soybeanragweed2/soybeanragweed2/3_dsm_ortho/2_mosaic/SoybeanRagweed2_transparent_mosaic_group1.tif"
with rasterio.open(raster_file) as src:
    # Read the data and metadata
    data = src.read(1, masked=True)
    meta = src.meta

    # Define the nodata value
    nodata = src.nodata

    # Mask out the nodata values
    masked_data = np.ma.masked_equal(data, nodata)

    # Calculate the area of valid pixels (excluding nodata)
    valid_pixels = np.count_nonzero(~masked_data.mask)

    # Get the resolution of the raster file
    x_res = meta['transform'][0]
    y_res = -meta['transform'][4]  # Negative because y-coordinate increases downward

    # Calculate the area in square meters
    area = valid_pixels * x_res * y_res

print("Area of the orthomosaic (excluding nodata):", area/4047, "acres")


Area of the orthomosaic (excluding nodata): 0.8590347556115644 acres


In [19]:
model = YOLO('best.pt').to('cuda')

In [20]:
model.device

device(type='cuda', index=0)

In [22]:
image = Image.open("C:/Users/rutvijw/Desktop/LabWork/geolocate/soybeanragweed2/soybeanragweed2/3_dsm_ortho/2_mosaic/SoybeanRagweed2_transparent_mosaic_group1.tif")
cnt = 0
width, height = image.size

In [ ]:
tile_size = 640
start_time = time.time()
for y in range(0, height, tile_size):
    for x in range(0, width, tile_size):
        # Calculate coordinates
        left = x
        upper = y
        right = min(x + tile_size, width)
        lower = min(y + tile_size, height)

        # Crop the tile
        tile = image.crop((left, upper, right, lower))
        # tile = TF.to_tensor(tile)[:3,:,:]
        # if tile.shape[1] != 640 or tile.shape[2] != 640:
        #     tile = TF.resize(tile, (640, 640))
        # result = model(tile.unsqueeze(0).to('cuda'))
        result = model(tile)
        cnt = cnt + 1
end_time = time.time()
print("done")


0: 640x640 1 Soybean, 15.6ms
Speed: 0.0ms preprocess, 15.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 6.0ms
Speed: 1.2ms preprocess, 6.0ms inference, 7.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 13.1ms
Speed: 1.8ms preprocess, 13.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 10.8ms
Speed: 3.4ms preprocess, 10.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 13.8ms
Speed: 2.5ms preprocess, 13.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 12.0ms
Speed: 2.8ms preprocess, 12.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 10.0ms
Speed: 2.9ms preprocess, 10.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 11.6ms
Speed: 2.9ms preprocess, 11.6ms inference, 2.0ms postprocess per image at shape (1,

In [11]:
elapsed_time = end_time - start_time

print("Time taken:", elapsed_time, "seconds")

Time taken: 149.8719139099121 seconds


In [21]:
149.8719139099121/1.3480080352792192 

111.18028230363511

In [22]:
640 * 640 * 2.1025e-06/4047

0.00021279565109957996

In [23]:
0.00021279565109957996/1.732

0.00012286123042700922

In [24]:
0.00012286123042700922 * 1000

0.12286123042700922

In [20]:
111.44178/0.8590347556115644

129.72907006616083

In [44]:
import torch
import torchvision.transforms.functional as TF
from torch.nn.functional import interpolate

times = []

tile_size = 640
batch_size = 32

alltiles = []
batch_tiles = []  # List to accumulate tiles into a batch

for y in range(0, height, tile_size):
    for x in range(0, width, tile_size):
        # Calculate coordinates
        left = x
        upper = y
        right = min(x + tile_size, width)
        lower = min(y + tile_size, height)

        # Crop the tile
        tile = image.crop((left, upper, right, lower))
        alltiles.append(tile)
        tile = TF.to_tensor(tile)[:3, :, :]
        # Resize if necessary
        if tile.shape[1] != 640 or tile.shape[2] != 640:
            tile = interpolate(tile.unsqueeze(0), size=(640, 640), mode='bilinear', align_corners=False)
            tile = tile.squeeze(0)

        # Add the tile to the batch list
        batch_tiles.append(tile)
        
        # Check if the batch is full
        if len(batch_tiles) == batch_size:
            # Stack the tiles along the batch dimension to create a batch tensor
            batch_tensor = torch.stack(batch_tiles, dim=0).to('cuda')
            start_time = time.time()
            # Process the batch with the model
            result = model(batch_tensor)
            end_time = time.time()

            times.append(end_time - start_time)
            
            # Clear the batch list for the next iteration
            batch_tiles.clear()


print("done")


0: 640x640 1 Soybean, 1: 640x640 1 Soybean, 2: 640x640 1 Soybean, 3: 640x640 1 Soybean, 4: 640x640 1 Soybean, 5: 640x640 1 Soybean, 6: 640x640 1 Soybean, 7: 640x640 1 Soybean, 8: 640x640 1 Soybean, 9: 640x640 1 Soybean, 10: 640x640 1 Soybean, 11: 640x640 1 Soybean, 12: 640x640 1 Soybean, 13: 640x640 1 Soybean, 14: 640x640 1 Soybean, 15: 640x640 1 Soybean, 16: 640x640 1 Soybean, 17: 640x640 1 Soybean, 18: 640x640 1 Soybean, 19: 640x640 1 Soybean, 20: 640x640 1 Soybean, 21: 640x640 1 Soybean, 22: 640x640 1 Soybean, 23: 640x640 1 Soybean, 24: 640x640 1 Soybean, 25: 640x640 1 Soybean, 26: 640x640 1 Soybean, 27: 640x640 1 Soybean, 28: 640x640 1 Soybean, 29: 640x640 1 Soybean, 30: 640x640 1 Soybean, 31: 640x640 1 Soybean, 100.2ms
Speed: 0.0ms preprocess, 3.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Soybean, 1: 640x640 2 Ragweeds, 1 Soybean, 2: 640x640 1 Soybean, 3: 640x640 1 Soybean, 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (

In [52]:
np.mean(np.array(times))

0.13491617980881118

In [46]:
np.mean(np.array(times))/(5.195206325673339e-10 * 640 * 640 * 32)

19.813048796999926

In [39]:
0.06929735698510166/0.0034047304175932797

20.35325811025187

In [30]:
elapsed_time/1.3480080352792192 ## This is for 16 batch size for 640 tile size

41.443080553415214

In [81]:
elapsed_time/1.3480080352792192 ## This is for 128 batch size for 640 tile size

41.74756367177438

In [86]:
elapsed_time/1.3480080352792192 ## This is for 64 batch size for 1280 tile size

22.503431646381788

In [91]:
elapsed_time/1.3480080352792192 ## This is for 128 batch size for 1280 tile size

22.882905627406867